# Starbucks Capstone Challenge

## Part I: Data Wrangling

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

The goal of this capstone project is to **implement a coupon recommender system** for Starbucks Mobile Rewards App, which uses customers' historical reactions to offers as well as their purchase behavior to generate customer clusters and model the most suitable offer for each cluster.

The challenge of **Part I: Data Wrangling** is that the provided transcript data set creates a new row for each interaction. Consequently, the first row will show the time a customer receives an offer. The next row will show the time when a customer views this or any other offer and the next row will show either the completion of an offer or a regular transaction. Furthermore, each customer can receive each offer multiple times or comlete an offer before he has seen it.

Therefore, the **goal of the data wrangling part is to create a master dataframe** (`df_mab`), which shows the time and amount of each purchase as well as the time of contact, view, and completion of the offer that influenced the purchase in a single row. The master dataframe should also contain purchases without offer infuence and offers that were not completed.

## Table of Content:
[1 Import Packages and Load Data](#1-bullet) <br>
[2 Editing Data](#2-bullet) <br>
&emsp;[2.1 Editing Portfolio-Data](#2.1-bullet) <br>
&emsp;[2.2 Editing Profile-Data](#2.2-bullet) <br>
&emsp;[2.3 Editing Transcript-Data](#2.3-bullet) <br>
[3 Creating a Master Dataframe for Offer-Interaction](#3-bullet) <br>
&emsp;[3.1 Table with received offers](#3.1-bullet) <br>
&emsp;[3.2 Table with viewed offers](#3.2-bullet) <br>
&emsp;[3.3 Table with completed offers](#3.3-bullet) <br>
&emsp;[3.4 Simplify combined Reaction-Dataframe](#3.4-bullet) <br>
&emsp;[3.5 Include Transactions with Offer Influence](#3.5-bullet) <br>
&emsp;[3.6 Include Transactions without Offer Influence](#3.6-bullet) <br>
&emsp;[3.7 Calculate Performance Variable](#3.7-bullet) <br>
&emsp;[3.8 Include Offers that did not have an Influence on Transactions](#3.8-bullet) <br>
&emsp;[3.9 Create a Dataframe with a Summary for each User](#3.2-bullet) <br>
[4 Verifying Results](#4-bullet) <br>


## 1 Import Packages and Load Data<a class="anchor" id="1-bullet"></a>

In [1]:
import pandas as pd
import numpy as np
import math
import json
import ast
%matplotlib inline
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
#read in the json files
portfolio = pd.read_json('../data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('../data/profile.json', orient='records', lines=True)
transcript = pd.read_json('../data/transcript.json', orient='records', lines=True)

## 2 Editing Data <a class="anchor" id="2-bullet"></a>

### 2.1 Editing Portfolio-Data <a class="anchor" id="2.1-bullet"></a>
Portfolio Data contains offer ids and meta data about each offer (duration, type, etc.)
- id (string) - offer id
- offer_type (string) - type of offer ie BOGO, discount, informational
- difficulty (int) - minimum required spend to complete an offer
- reward (int) - reward given for completing an offer
- duration (int) - time for offer to be open, in days
- channels (list of strings)

In [3]:
portfolio.head(1)

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd


Editing Steps:
- Rename `id` as `offer_id`
- Create `offer_name` based on `offer_type`, `difficulty`, `reward` and `duration`
- Create Dummi-Variables for column `channels` 
- Create Dummi-Variables for column `offer_type` 

In [5]:
# rename id as offer_id
portfolio = portfolio.rename({'id': 'offer_id'}, axis = 1)

#create offer names
portfolio['offer_name'] = portfolio['offer_type'].str[:4] \
                            + '_df' + portfolio['difficulty'].astype(str) \
                            + '_rw' + portfolio['reward'].astype(str) \
                            + '_du' + portfolio['duration'].astype(str)

# convert channels to dummies
pf_channels = portfolio['channels'].astype(str).str.replace(r"[\[\]']", '')\
                                               .str.get_dummies(sep=', ')

# merge channel-dummies to df
if not 'email' in portfolio.columns:
    portfolio = pd.concat([portfolio, pf_channels], axis=1)
portfolio = portfolio.drop('channels', axis = 1)

#create dummies from offer type
portfolio['offer_type2'] = portfolio['offer_type']
portfolio = pd.get_dummies(portfolio, prefix = '', prefix_sep = '', columns=['offer_type2'])

portfolio = portfolio.sort_values(by = ['offer_type','difficulty','reward','duration'], ascending = True)\
                     .reset_index(drop = True)

print(portfolio.dtypes)
portfolio.head(15)

reward            int64
difficulty        int64
duration          int64
offer_type       object
offer_id         object
offer_name       object
email             int64
mobile            int64
social            int64
web               int64
bogo              uint8
discount          uint8
informational     uint8
dtype: object


C:\Users\fisc313\AppData\Local\Temp\ipykernel_23780\2575384851.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  pf_channels = portfolio['channels'].astype(str).str.replace(r"[\[\]']", '')\


,reward,difficulty,duration,offer_type,offer_id,offer_name,email,mobile,social,web,bogo,discount,informational
0,5,5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d,bogo_df5_rw5_du5,1,1,1,1,1,0,0
1,5,5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo_df5_rw5_du7,1,1,0,1,1,0,0
2,10,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo_df10_rw10_du5,1,1,1,1,1,0,0
3,10,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,bogo_df10_rw10_du7,1,1,1,0,1,0,0
4,3,7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2,disc_df7_rw3_du7,1,1,1,1,0,1,0
5,2,10,7,discount,2906b810c7d4411798c6938adc9daaa5,disc_df10_rw2_du7,1,1,0,1,0,1,0
6,2,10,10,discount,fafdcd668e3743c1bb461111dcafc2a4,disc_df10_rw2_du10,1,1,1,1,0,1,0
7,5,20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,disc_df20_rw5_du10,1,0,0,1,0,1,0
8,0,0,3,informational,5a8bc65990b245e5a138643cd4eb9837,info_df0_rw0_du3,1,1,1,0,0,0,1
9,0,0,4,informational,3f207df678b143eea3cee63160fa8bed,info_df0_rw0_du4,1,1,0,1,0,0,1


### 2.2 Editing Profile-Data  <a class="anchor" id="2.2-bullet"></a>
Profile Data contains demographic data for each customer.

- gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
- age (int) - age of the customer
- id (str) - customer id
- became_member_on (int) - date when customer created an app account
- income (float) - customer's income

In [6]:
profile.head(1)

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN


Editing Steps:
- Rename `id` as `member_id` 
- Converting `age`==118 to 'NaN'
- Converting `became_member_on` to a date

In [7]:
#rename id as member_id
profile = profile.rename({'id': 'member_id'}, axis = 1)

#convert age = 118 to NaN
profile['age'] = profile['age'].apply(lambda x: np.nan if x == float(118) else float(x))

#calculate registration date
profile['reg_dt'] = pd.to_datetime(profile['became_member_on'], format='%Y%m%d')

#calculate days since registration (assuming todays date is '2018-07-27')
profile['days_reg'] = (datetime(2018,7,27) - profile['reg_dt']).dt.days

print(profile.dtypes)
profile.head(2)

gender                      object
age                        float64
member_id                   object
became_member_on             int64
income                     float64
reg_dt              datetime64[ns]
days_reg                     int64
dtype: object


,gender,age,member_id,became_member_on,income,reg_dt,days_reg
0,None,NaN,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN,2017-02-12,530
1,F,55.0,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0,2017-07-15,377


### 2.3 Editing Transcript-Data <a class="anchor" id="2.3-bullet"></a>
Trascript Data contains records for transactions, offers received, offers viewed, and offers completed.
- person (str) - customer id
- event (str) - record description (ie transaction, offer received, offer viewed, etc.)
- value - (dict of strings) - either an offer id or transaction amount depending on the record
- time (int) - time in hours since start of test. The data begins at time t=0

In [8]:
transcript.head(1)

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0


Editing Steps:
- Rename `person` as `member_id` 
- Converting `value`-dictionary to individual columns

In [ ]:
#rename person as member_id
transcript = transcript.rename({'person': 'member_id'}, axis = 1)

#check columns of dataset
print('Number Rows: ' + str(len(transcript)))
print('Unique Members: ' + str(transcript['member_id'].nunique()))
print('NULLs per Column: ')
print(np.sum(transcript.isna()))
print('Values of Event Column: ' + str(transcript['event'].explode().unique()))

#convert value-dictionary to individual columns
transcript['value'] = transcript['value'].astype(str).str.replace('r i', 'r_i')
transcript['value'] = transcript['value'].apply(lambda x: ast.literal_eval(x))
transcript = pd.concat([transcript.drop(['value'], axis=1), transcript['value'].apply(pd.Series)], axis=1)

#view transcript of one member
transcript[transcript['member_id'] == 'ffff82501cea40309d5fdd7edcca4a07']

Number Rows: 306534
Unique Members: 17000
NULLs per Column: 
member_id    0
event        0
value        0
time         0
dtype: int64
Values of Event Column: ['offer received' 'offer viewed' 'transaction' 'offer completed']


## 3 Creating a Master Dataframe for Offer-Interaction<a class="anchor" id="3-bullet"></a>
Instead of listing each interaction with an offer in a new row, the relevant information should be found in only one row.

### 3.1 Table with received offers<a class="anchor" id="3.1-bullet"></a>

In [ ]:
#Dataframe of received offers
rec_offer = transcript[transcript['event'] == 'offer received']
rec_offer = rec_offer.drop(['amount','reward'], axis = 1)

#Merge offer information on received offers
if not 'offer_type' in rec_offer.columns:
    rec_offer = pd.merge(rec_offer, portfolio, on = 'offer_id', how = 'left')
    print('Merge completed')

#create a rank per member and coupon
rec_offer['offer_rnk'] = rec_offer.sort_values(['time'], ascending=[True]) \
                           .groupby(['member_id','offer_id']) \
                           .cumcount() + 1

#calculate ending date of offer
rec_offer['offer_valid_to'] = rec_offer['time'] + (rec_offer['duration'] * 24)

### 3.2 Table with viewed offers <a class="anchor" id="3.2-bullet"></a>

In [ ]:
#Dataframe of viewed offers
view_offer = transcript[transcript['event'] == 'offer viewed']
view_offer = view_offer.drop(['event','amount','reward'], axis = 1)
view_offer = view_offer.rename({'time': 'time_viewed'}, axis = 1)

#Merge viewed offers with recieved offers
offer_react = pd.DataFrame()
if not 'offer_viewed' in offer_react.columns:
    offer_react = pd.merge(rec_offer, view_offer, on = ['offer_id','member_id'], how = 'left')
    print('Merge completed')
    
#include only rows where view_time is between rceived_time and offer_valid_to
#NaNs should be included as well
offer_react['valid_viewed'] = np.where(offer_react['time_viewed'].isnull(), 1, None)
offer_react['valid_viewed'] = np.where((offer_react['offer_valid_to'] >= offer_react['time_viewed']) \
                                       & (offer_react['time'] <= offer_react['time_viewed']), 1, offer_react['valid_viewed'])
offer_react['valid_viewed'] = offer_react['valid_viewed'].fillna(0)

#create a rank per member and coupon
offer_react['viewed_rnk'] = offer_react.sort_values(['valid_viewed'], ascending=[False]) \
                           .groupby(['member_id','offer_id','time']) \
                           .cumcount() + 1

offer_react['time_viewed_valid'] = np.where((offer_react['valid_viewed'] == 0) & (offer_react['viewed_rnk'] == 1), np.NaN, offer_react['time_viewed'])

offer_react = offer_react[ ((offer_react['valid_viewed'] == 1) & (offer_react['viewed_rnk'] == 1) ) | ((offer_react['viewed_rnk'] == 1))]

### 3.3 Table with completed offers <a class="anchor" id="3.3-bullet"></a>

In [ ]:
#Dataframe of viewed offers
compl_offer = transcript[transcript['event'] == 'offer completed']
compl_offer = compl_offer.drop(['event','amount','reward'], axis = 1)
compl_offer = compl_offer.rename({'time': 'time_compl'}, axis = 1)

#Merge completed offers with recieved offers
if not 'time_compl' in offer_react.columns:
    offer_react = pd.merge(offer_react, compl_offer, on = ['offer_id','member_id'], how = 'left')
    print('Merge completed')
    
#include only rows where time_compl is between received_time and offer_valid_to
#NaNs should be included as well
offer_react['valid_compl'] = np.where(offer_react['time_compl'].isnull(), 1, None)
offer_react['valid_compl'] = np.where((offer_react['offer_valid_to'] >= offer_react['time_compl']) \
                                       & (offer_react['time'] <= offer_react['time_compl']), 1, offer_react['valid_compl'])
offer_react['valid_compl'] = offer_react['valid_compl'].fillna(0)

#create a rank per member and coupon
offer_react['compl_rnk'] = offer_react.sort_values(['valid_compl'], ascending=[False]) \
                                      .groupby(['member_id','offer_id','time']) \
                                      .cumcount() + 1

offer_react['time_comp_valid'] = np.where((offer_react['valid_compl'] == 0) & (offer_react['compl_rnk'] == 1), np.NaN, offer_react['time_compl'])

#only keep valid rows
offer_react = offer_react[ ((offer_react['valid_compl'] == 1) & (offer_react['compl_rnk'] == 1) ) \
                          | ((offer_react['compl_rnk'] == 1))]

### 3.4 Simplify combined Reaction-Dataframe <a class="anchor" id="3.4-bullet"></a>

In [ ]:
pd.set_option('display.max_columns', None)

#drop unnecessary columns
reaction = offer_react.copy()
reaction = reaction.drop(['event','offer_rnk','time_viewed','valid_viewed','viewed_rnk','time_compl','valid_compl','compl_rnk'], axis = 1)
#rename columns
reaction = reaction.rename({'time': 'time_rec','time_viewed_valid': 'time_viewed', 'time_comp_valid': 'time_compl'}, axis = 1)

#create dummy variable for offer influence: 
#1 = 'the offer had an influence on a transaction'
#0 = 'no influence'
#an offer must be completed and viewed befor completion to have an influnence
#exception: informational offer types only have to be viewed to have an influence
reaction['offer_incluence'] = np.where(reaction['time_compl']>=reaction['time_viewed'], 1, np.NaN)
reaction['offer_incluence'] = np.where(reaction['time_viewed'].isnull(), 0, reaction['offer_incluence'])
reaction['offer_incluence'] = np.where(reaction['time_compl'].isnull(), 0, reaction['offer_incluence'])
reaction['offer_incluence'] = np.where((reaction['offer_type'] == 'informational') & (reaction['time_viewed']>=0 ), 1, reaction['offer_incluence'])
reaction['offer_incluence'] = reaction['offer_incluence'].fillna(0)

#create further variables
reaction['viewed'] = np.where(reaction['time_viewed'] >= reaction['time_rec'], 1, 0)
reaction['hours_viewed'] = np.where(reaction['viewed'] == 1, reaction['time_viewed'] - reaction['time_rec'], np.NaN)
reaction['hours_compl'] = np.where(reaction['offer_incluence'] == 1, reaction['time_compl'] - reaction['time_viewed'], np.NaN)
reaction['completed'] = np.where(reaction['hours_compl'].isnull(), 0, 1)

### 3.5 Include Transactions with Offer Influence  <a class="anchor" id="3.5-bullet"></a>
So far, the reaction dataframe contains all received offers for each customer. In the next step, regular transactions will be merged with the offer reactions. However, due to the data structure at first only transactions that are influenced by an offer can be merged. Transactions without offer influence have to be merged in a separate step.

In [ ]:
#Dataframe of transactions
transaction = transcript[transcript['event'] == 'transaction']
transaction = transaction.drop(['event','offer_id','reward'], axis = 1)

#Create Sub-Dataframe of reation, that only holds offers with an influence on transactions
#reaction['offer_incluence']: offers that have an influence on transactions
r = reaction[['member_id','time_rec','offer_name','viewed','time_viewed','hours_viewed','completed','time_compl','hours_compl','offer_valid_to','bogo','discount','informational','reward','difficulty','duration']]\
            [reaction['offer_incluence']==1]
r['offer_end'] = np.where(r['time_compl'].isnull(),r['offer_valid_to'],r['time_compl'])

#Merge influential offers with transactions
if not 'time_compl' in transaction.columns:
    transaction = pd.merge(transaction, r, on = ['member_id'], how = 'left')
    print('Merge completed')

#Calculate new offer influence column for transactions   
#transaction['offer_influence']: transactions that are influenced by an offer
transaction['offer_influence'] = 0 
transaction['offer_influence'] = np.where((transaction['time'] <= transaction['offer_end']) & (transaction['time'] >= transaction['time_viewed']), 1, transaction['offer_influence'])

#drop rows without offer influence
transaction['valid'] = np.where((transaction['offer_influence'] == 1),1,np.NaN)
transaction = transaction[transaction['valid']==1]

#What is the influential offer type?
transaction['bogo'] = np.where(transaction['offer_influence'] == 0, 0, transaction['bogo'])
transaction['discount'] = np.where(transaction['offer_influence'] == 0, 0, transaction['discount'])
transaction['informational'] = np.where(transaction['offer_influence'] == 0, 0, transaction['informational'])

#Offer type 'informational' should be 'completed' if a transaction occured during its validity period
transaction['completed'] = np.where(transaction['informational']==1,1,transaction['completed'])
transaction['time_compl'] = np.where(transaction['informational']==1,transaction['time'],transaction['time_compl'])
transaction['hours_compl'] = np.where(transaction['informational']==1,transaction['time_compl']-transaction['time_viewed'],transaction['hours_compl'])

### 3.6 Include Transactions without Offer Influence <a class="anchor" id="3.6-bullet"></a>

In [ ]:
#Dataframe of transactions
transaction_fin = transcript[transcript['event'] == 'transaction']
transaction_fin = transaction_fin.drop(['event','offer_id','reward'], axis = 1)

#Merge influenced transactions with all transactions
if not 'time_compl' in transaction_fin.columns:
    transaction_fin = pd.merge(transaction_fin, transaction, on = ['member_id','time','amount'], how = 'left')
    print('Merge completed')
transaction_fin.sort_values(by = ['member_id','time'], ascending = True).head(20)

#adapt columns for transaction without coupon offer
transaction_fin['offer_transaction'] = np.where(transaction_fin['offer_name'].isnull(),'without_offer','with_offer')
transaction_fin['offer_name'] = np.where(transaction_fin['offer_name'].isnull(),'No_offer',transaction_fin['offer_name'])
transaction_fin['viewed'] = np.where(transaction_fin['offer_name']=='No_offer',1,transaction_fin['viewed'])
transaction_fin['completed'] = np.where(transaction_fin['offer_name']=='No_offer',1,transaction_fin['completed'])
transaction_fin['bogo'] = transaction_fin['bogo'].fillna(0)
transaction_fin['discount'] = transaction_fin['discount'].fillna(0)
transaction_fin['informational'] = transaction_fin['informational'].fillna(0)
transaction_fin['reward'] = transaction_fin['reward'].fillna(0)
transaction_fin['difficulty'] = transaction_fin['difficulty'].fillna(0)
transaction_fin['duration'] = transaction_fin['duration'].fillna(1)
transaction_fin['time_rec'] = transaction_fin['time_rec'].fillna(transaction_fin['time'])

#create a rank per member, coupon, and rec_dt
transaction_fin['offer_compl_rnk'] = transaction_fin.sort_values(['time'], ascending=[True]) \
                                                    .groupby(['member_id','offer_name','time_rec']) \
                                                    .cumcount() + 1
transaction_fin['offer_compl_rnk'] = np.where(transaction_fin['offer_name']=='No_offer',0,transaction_fin['offer_compl_rnk'])
transaction_fin['offer_compl_rnk'] = np.where(transaction_fin['offer_compl_rnk']>=2,0,transaction_fin['offer_compl_rnk'])

#informational offers
transaction_fin['hours_compl'] = np.where((transaction_fin['informational']==1) & (transaction_fin['offer_compl_rnk'] == 0), np.nan, transaction_fin['hours_compl'])
transaction_fin['offer_influence'] = np.where((transaction_fin['informational']==1) & (transaction_fin['offer_compl_rnk'] == 0), np.nan, transaction_fin['offer_influence'])
transaction_fin['valid'] = np.where((transaction_fin['informational']==1) & (transaction_fin['offer_compl_rnk'] == 0), np.nan, transaction_fin['valid'])
transaction_fin['hours_compl'] = np.where((transaction_fin['informational']==1) & (transaction_fin['offer_compl_rnk'] == 0), np.nan, transaction_fin['hours_compl'])
transaction_fin['completed'] = np.where((transaction_fin['informational']==1) & (transaction_fin['offer_compl_rnk'] == 0), 0, transaction_fin['completed'])
transaction_fin['offer_transaction'] = np.where((transaction_fin['informational']==1) & (transaction_fin['offer_compl_rnk'] == 0), 'without_offer', transaction_fin['offer_transaction'])
transaction_fin['offer_name'] = np.where((transaction_fin['informational']==1) & (transaction_fin['offer_compl_rnk'] == 0), 'No_offer', transaction_fin['offer_name'])

#Was the following transaction made with or without an offer?
transaction_fin['next_transaction'] = transaction_fin.groupby(['member_id'])['offer_transaction'].shift(-1)
transaction_fin['next_transaction_num'] = np.where(transaction_fin['next_transaction'] == 'without_offer',1,0)
transaction_fin['next_transaction_num'] = np.where(transaction_fin['next_transaction'] == 'with_offer',1,transaction_fin['next_transaction_num'])

#How many hours have past since the last transaction?
transaction_fin['last_transaction_time_diff'] = transaction_fin['time'] - transaction_fin.groupby(['member_id'])['time'].shift(1)
transaction_fin['last_transaction_time_diff'] = transaction_fin['last_transaction_time_diff'].fillna(999)
transaction_fin['last_transaction_time_diff'] = np.where(transaction_fin['last_transaction_time_diff'] <=12,12, transaction_fin['last_transaction_time_diff'])

#Wat was the mean amout of the 3 transactions before? And what is the difference to the current transaction amount?
transaction_fin['mean_amount'] = round(transaction_fin.groupby('member_id')['amount'].transform(lambda x: x.rolling(3, 1).mean()),2)
transaction_fin['last_transactions_amount_diff'] = transaction_fin['mean_amount'] - transaction_fin.groupby(['member_id'])['mean_amount'].shift(1)
transaction_fin['last_transactions_amount_diff'] = transaction_fin['last_transactions_amount_diff'].fillna(0)
transaction_fin['last_transactions_amount_diff'] = np.where(transaction_fin['last_transactions_amount_diff'] >=3,3, transaction_fin['last_transactions_amount_diff'])
transaction_fin['last_transactions_amount_diff'] = np.where(transaction_fin['last_transactions_amount_diff'] <=-3,-3, transaction_fin['last_transactions_amount_diff'])

#How is the relation between the reward a user receives with offer completion and the offers' difficulty?
transaction_fin['costs'] = np.where(transaction_fin['difficulty'] > 0, transaction_fin['reward']/transaction_fin['difficulty'],0)

### 3.7 Calculate Performance Variable <a class="anchor" id="3.7-bullet"></a>
The performance variable will be the target variable that should be optimized in the multi-armed bandit model. It will be calculated as followed: 
* 3 Points if amount of current transaction is higher than average of the last 3 transactions
* 1 Point if next transaction is made
* 1 Point if time until next transaction is <= 48 (Median of last_transaction_time_diff: 48 hours = 2 days) 
* 1 Point of Offer is completed within 48 hours

Minimum Performance is 0 and maximum Performance is 4

In [23]:
# 3 Points if amount of current transaction is higher than average of the last 3 transactions
transaction_fin['performance'] = np.where(transaction_fin['last_transactions_amount_diff'] > 0, 3, 0)
# 1 Point if next transaction is made with an offer
transaction_fin['performance'] = transaction_fin['performance'] + transaction_fin['next_transaction_num']
# 1 Point of Offer is completed within 48 hours
transaction_fin['performance'] = transaction_fin['performance'] + np.where((transaction_fin['hours_compl'] >= 0) & (transaction_fin['hours_compl'] <= 48), 1, 0)
# 1 Point if time until next transaction is <= 48 (Median of last_transaction_time_diff: 48 hours = 2 days) 
transaction_fin['performance'] = transaction_fin['performance'] + np.where(transaction_fin['last_transaction_time_diff'] <= 48, 1, 0)
#maximum performance should be 4
transaction_fin['performance'] = np.where(transaction_fin['performance']==4, 3, transaction_fin['performance'])
transaction_fin['performance'] = np.where(transaction_fin['performance']>=5, 4, transaction_fin['performance'])

# (minus costs: transaction_fin['reward']/transaction_fin['difficulty'])
#transaction_fin['performance'] = np.where(transaction_fin['performance']>=transaction_fin['costs']\
#                                          ,transaction_fin['performance'] - transaction_fin['costs']\
#                                          ,transaction_fin['performance'])

### 3.8 Include Offers that did not have an Influence on Transactions <a class="anchor" id="3.8-bullet"></a>

In [ ]:
#get distinct influential offers from transaction dataframe
used = transaction_fin[['member_id','time_rec','offer_name']][transaction_fin['offer_name'] != 'No_offer'].copy()
used.drop_duplicates(inplace=True)
used['used']=1

#merge them with all offers
not_used = reaction[['member_id','time_rec','offer_name','viewed','time_viewed','hours_viewed','completed','time_compl','hours_compl','offer_valid_to','bogo','discount','informational','reward','difficulty','duration']]
if not 'used' in not_used.columns:
    not_used = pd.merge(not_used, used, on = ['member_id','time_rec','offer_name'], how = 'left')
    print('Merge durchgeführt')
    
#delete rows that where in transaction dataframe as well as offers that were not viewed
not_used = not_used[(not_used['used'] != 1) & (not_used['viewed']==1)]
not_used = not_used.rename({'time_rec': 'time'}, axis = 1)
not_used['amount'] = 0

#create a rank per member and not used coupon
not_used['NU_offer_rnk'] = not_used.sort_values(['time'], ascending=[True]) \
                           .groupby(['member_id','offer_name']) \
                           .cumcount() + 1

#if rank = 1 then performance = 1 else perfomance = 0, to give the not used offer a better chance to be tried a second time
not_used['performance'] = np.where(not_used['NU_offer_rnk'] == 1,1,0)
not_used = not_used[['member_id','time','amount','offer_name','viewed','completed','performance','bogo','discount','informational','reward','difficulty','duration']]

#create master dataframe
df_mab = transaction_fin[['member_id','time','amount','offer_name','viewed','completed','offer_compl_rnk','performance','bogo','discount','informational','reward','difficulty','duration']]
df_mab = pd.concat([df_mab, not_used])

### 3.9 Create a Dataframe with a Summary for each User  <a class="anchor" id="3.9-bullet"></a>

In [ ]:
#aggregate transactional data by user
df_member = transaction_fin.groupby(['member_id'])\
                           .aggregate({'completed': ['sum'],\
                                       'offer_influence': ['sum'],\
                                       'amount': ['sum','median','std'],\
                                       'offer_compl_rnk': ['sum'],\
                                       'bogo': ['sum'],\
                                       'discount': ['sum'],\
                                       'informational': ['sum'],\
                                       'hours_compl': ['mean']})\
                           .reset_index()
df_member.columns = ['member_id', 'nr_purch','nr_purch_offer','amount_sum','amount_median','amount_std','compl_offers','used_bogo','used_disc','used_info','mean_hrs_compl']

#aggregate offers that were not used by the user
agg_not_used =     not_used.groupby(['member_id'])\
                           .aggregate({'offer_name': ['count','nunique'],\
                                       'viewed': ['sum'],\
                                       'bogo': ['sum'],\
                                       'discount': ['sum'],\
                                       'informational': ['sum']})\
                           .reset_index()
agg_not_used.columns = ['member_id','nr_NUO','nr_dist_NUO','sum_viewed_NUO','nr_bogo_NUO','nr_disc_NUO','nr_info_NUO']

#Merge not used offers on df_member
if not 'nr_NUO' in df_member.columns:
    df_member = pd.merge(df_member, agg_not_used, on = 'member_id', how = 'left')
    print('Merge 1 completed')

#Merge days since registration on df_member
if not 'days_reg' in df_member.columns:
    df_member = pd.merge(df_member, profile[['member_id','days_reg']], on = 'member_id', how = 'left')
    print('Merge 2 completed')

## 4 Verifying Results  <a class="anchor" id="4-bullet"></a>

Compare original input dataframe with final master dataframes:

In [ ]:
transcript[transcript['member_id'] == 'fff3ba4757bd42088c044ca26d73817a']

In [ ]:
df_mab[df_mab['member_id'] == 'fff3ba4757bd42088c044ca26d73817a']

In [ ]:
df_member[df_member['member_id'] == 'fff3ba4757bd42088c044ca26d73817a']

**Comment:** The original `transcript` shows that the user received and comleted the offer *'bogo_df5_rw5_du7'*. However, in the final `df_map` this offer is listed as not completed. Since the user completed the offer before he/she viewed it, the final dataframe does not consider the offer as infuential and the trasaction at the time 168 is categorised as *'No_offer'*-influence. During the hour 336 the user received a *'info_df0_rw0_du3'* offer which he/she never viewed and with the hour 408 he/she received a *'disc_df10_rw2_du7'* offer which was completed but never viewed. Because both offers were not viewed by the customer, they listed in the final dataframe at all. The recommendation system should only consider the performance of offers that have been viewed by the customers.

In [ ]:
#save dataframe
df_member.to_csv('../data/df_member.csv', sep='\t', index=False)
df_mab.to_csv('./data/df_mab.csv', sep='\t', index=False)